In [1]:
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

import pandas as pd
import time
import numpy as np
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

In [2]:
house_df = pd.read_csv('data/house_mean_imputation.csv')
#house_df = pd.read_csv('data/ready_to_use_dataset.csv')
house_df = house_df.drop_duplicates(['year', 'state', 'district', 'name'])

display(house_df.shape)

(9802, 19)

In [3]:
house_df['last_house_majority'].unique() 

array(['R', 'D'], dtype=object)

In [9]:
#house_df2=house_df.set_index(['year', 'state', 'district', 'name']).dropna().copy()
house_df2=house_df.dropna().copy()
house_df2_districts=house_df2[['state','district']]
house_df2=house_df2.drop('state', axis=1).drop('district', axis=1).drop('name', axis=1)
house_df2['party']=house_df2['party'].replace(['D', 'R'], [0, 1])
house_df2['president_party']=house_df2['president_party'].replace(['D', 'R'], [0, 1])
house_df2['last_house_majority']=house_df2['last_house_majority'].replace(['D', 'R'], [0, 1])
data_train, data_test=house_df2[house_df2['year']!=2018], house_df2[house_df2['year']==2018]

x_train, y_train=data_train.drop('won', axis=1), data_train['won']

x_test, y_test=data_test.drop('won', axis=1), data_test['won']
baselineLogRegr=LogisticRegressionCV(cv=5, penalty='l2').fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [5]:
display(house_df[(house_df['year']!=2018)&(house_df['state']=='Alabama')&(house_df['district']=='District 1')])

,district,is_incumbent,name,party,percent,state,votes,won,year,count_victories,unemployement_rate,is_presidential_year,president_can_be_re_elected,president_party,president_overall_avg_job_approval,last_D_house_seats,last_R_house_seats,last_house_majority,fundraising
4192,District 1,0.0,Judy Belk,D,38.0,Alabama,67507,0,2002,0,5.7,0.0,1.0,R,0.494,212.0,221.0,R,7.465678e+05
4193,District 1,0.0,Jo Bonner,R,61.0,Alabama,108102,1,2002,0,5.7,0.0,1.0,R,0.494,212.0,221.0,R,7.465678e+05
4945,District 1,0.0,Judy Belk,D,37.0,Alabama,93938,0,2004,0,5.5,1.0,0.0,R,0.494,205.0,229.0,R,7.465678e+05
4946,District 1,1.0,Jo Bonner,R,63.0,Alabama,161067,1,2004,1,5.5,1.0,0.0,R,0.494,205.0,229.0,R,7.465678e+05
5469,District 1,1.0,Jo Bonner,R,68.0,Alabama,112944,1,2006,2,4.4,0.0,0.0,R,0.494,202.0,232.0,R,7.465678e+05
5489,District 1,0.0,Vivian Beckerle,D,32.0,Alabama,52770,0,2006,0,4.4,0.0,0.0,R,0.494,202.0,232.0,R,7.465678e+05
5649,District 1,1.0,Jo Bonner,R,98.0,Alabama,210660,1,2008,3,6.5,1.0,1.0,D,0.480,233.0,202.0,D,7.465678e+05
6207,District 1,1.0,Jo Bonner,R,83.0,Alabama,129063,1,2010,4,9.4,0.0,1.0,D,0.480,257.0,178.0,D,9.001988e+05
7202,District 1,1.0,Jo Bonner,R,97.9,Alabama,196374,1,2012,5,10.9,1.0,0.0,D,0.480,193.0,242.0,R,1.177293e+06
8086,District 1,0.0,Burton LeFlore,D,31.7,Alabama,48278,0,2014,0,8.3,0.0,0.0,D,0.480,201.0,234.0,R,4.001050e+03


In [11]:
#Accuracy is defined as (TP+TN)/n
def printAccuracy(y_train, y_pred_train, y_test, y_pred_test):
    print('Training Set Accuracy: \t{:.2%}'.format(np.sum(y_train == y_pred_train) / len(y_train)))
    print('Test Set Accuracy: \t{:.2%}'.format(np.sum(y_test == y_pred_test) / len(y_test)))

y_pred_train=baselineLogRegr.predict_proba(x_train)[:,1]
#y_pred_train=baselineLogRegr.predict(x_train)
y_pred_test=baselineLogRegr.predict_proba(x_test)[:,1]
display(y_pred_train)

print('Amount of districts in the predictions: {:.1%} of the total'.format(len(x_test.join(house_df2_districts).groupby(['state', 'district']).count())/435))

array([1.43009537e-01, 1.73243987e-01, 7.48997149e-01, ...,
       9.14887751e-01, 9.90917812e-01, 5.95622315e-06])

Amount of districts in the predictions: 98.4% of the total


In [ ]:
#Add index to predictions from X set
y_pred_train=pd.DataFrame(y_pred_train, index=x_train.index, columns=['y_pred'])
y_pred_test=pd.DataFrame(y_pred_test, index=x_test.index, columns=['y_pred'])

#Join candidates data by index
house_df2_pred_districts=house_df2_candidates.join(y_pred_test)
display(house_df2_pred_districts.head())

#Group by district
house_df2_pred_districts.groupby(['state', 'district']).agg({'y_pred':('max', 'mean')})
house_df2_pred_districts

In [ ]:
printAccuracy(y_train, y_pred_train, y_test, y_pred_test)